# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
read_path = os.path.join("output_data/cities.csv")
main_frame = pd.read_csv(read_path, low_memory=False)
main_frame.head()

,City_ID,Country,Date,Latitude,Longitude,Cloudiness,Humidity,Max Temp,Wind Speed
0,0,AU,1584212918,-33.65,115.33,83,98,66.00,13.27
1,1,US,1584212749,37.10,-113.58,75,54,61.00,13.87
2,2,ZA,1584212935,-34.42,19.23,26,88,64.99,1.01
3,3,AR,1584212845,-54.80,-68.30,75,58,53.60,10.29
4,4,NZ,1584212826,-46.19,168.86,85,85,51.01,1.01


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
type(main_frame['Humidity'][1])

numpy.int64

In [4]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# store locations
locations = main_frame[['Latitude', 'Longitude']]

# Fill NaN values and convert to float
humidity = main_frame["Humidity"].astype(float)

In [5]:
main_frame["Max Temp"].max()

104.0

In [6]:
# plot gmaps
figure = gmaps.figure() # this is to make a base gmap

# add humidity layer on top of gmap
humidity_layer = gmaps.heatmap_layer(locations,
                                     weights = humidity,  # == weights
                                     dissipating = True,
                                     max_intensity = 105,
                                     point_radius = 6
                                     )
figure.add_layer(humidity_layer)
figure

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [18]:
# filter weather conditions:
maxtemp_df = main_frame.loc[(main_frame['Max Temp'] < 80) &
                             (main_frame['Max Temp'] > 70) &
                             (main_frame['Cloudiness'] == 0) &
                             (main_frame['Wind Speed'] < 10),
                              :]
# # maxtemp_df = maxtemp_df.loc[(main_frame['Cloudiness'] == 0), :]
maxtemp_df

,City_ID,Country,Date,Latitude,Longitude,Cloudiness,Humidity,Max Temp,Wind Speed
157,157,LK,1584212961,5.95,80.54,0,83,74.82,5.73
196,196,IN,1584212968,25.75,71.38,0,25,71.04,8.48
220,220,ID,1584212972,2.05,99.75,0,96,74.46,1.95
278,278,LK,1584212981,6.12,81.12,0,88,78.80,2.24
358,358,ID,1584212995,4.47,97.97,0,88,76.98,2.77
427,427,PK,1584213005,28.52,68.54,0,29,70.32,6.49
537,537,AR,1584213025,-39.03,-67.58,0,11,71.60,8.05


In [17]:
main_frame['Cloudiness'].describe()

count    569.000000
mean      54.506151
std       37.201115
min        0.000000
25%       20.000000
50%       67.000000
75%       90.000000
max      100.000000
Name: Cloudiness, dtype: float64

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

NameError: name 'narrowed_city_df' is not defined

In [ ]:
# Add marker layer ontop of heat map


# Display Map